In [ ]:
pip install opencv-python matplotlib

In [1]:
import tensorflow as tf
import os
import cv2
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dataDir='/content/drive/MyDrive/Data'
train_folder ='/content/drive/MyDrive/Data/train'
valid_folder = '/content/drive/MyDrive/Data/valid'
test_folder = '/content/drive/MyDrive/Data/test'
os.listdir(dataDir)

['test', 'train', 'valid']

In [3]:
Cclasses = os.listdir(test_folder)
Cclasses

['adenocarcinoma', 'large.cell.carcinoma', 'normal', 'squamous.cell.carcinoma']

In [14]:
from PIL import Image


def filter_images(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            file_path = os.path.join(root, filename)

            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                try:

                    with Image.open(file_path) as img:
                        img.verify()
                except OSError:

                    print(f"Removing: {file_path}")
                    os.remove(file_path)
            else:

                print(f"Removing: {file_path} - Unsupported file type")
                os.remove(file_path)

filter_images(test_folder)
filter_images(train_folder)
filter_images(valid_folder)


Removing: /content/drive/MyDrive/Data/train/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/000041 (7).png
Removing: /content/drive/MyDrive/Data/train/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/000041 (4).png
Removing: /content/drive/MyDrive/Data/train/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/000041 (2).png
Removing: /content/drive/MyDrive/Data/train/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa/000026 (2).png
Removing: /content/drive/MyDrive/Data/train/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa/000027.png
Removing: /content/drive/MyDrive/Data/train/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa/000031.png
Removing: /content/drive/MyDrive/Data/train/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa/000026.png
Removing: /content/drive/MyDrive/Data/train/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa/000003 (3).png
Removing: /content/drive/MyDrive/Data/train/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa/000027 (2).png
Removing: /content/drive/MyDrive/Data/train/large.cell.carcinoma_left.hilum_

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from keras.utils import to_categorical
from sklearn.utils import shuffle

#buiding generators
basic_gen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    dtype = 'float32'
)
train_gen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range = 10,
    width_shift_range = 0.25,
    height_shift_range = 0.25,
    zoom_range = 0.1,
    horizontal_flip = True,
    vertical_flip = True,
    dtype = 'float32'
)
#data gen
train_data = train_gen.flow_from_directory(
    train_folder,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'categorical'
)

valid_data = basic_gen.flow_from_directory(
    valid_folder,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'categorical'
)

test_data = basic_gen.flow_from_directory(
    test_folder,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'categorical',
    shuffle = False
)



Found 598 images belonging to 4 classes.
Found 72 images belonging to 4 classes.
Found 297 images belonging to 4 classes.


In [12]:
from tensorflow.keras.applications import MobileNetV2
from keras.src.metrics.confusion_metrics import activations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,MaxPooling2D,Flatten,BatchNormalization,Dropout
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#freezing layer to use the model as feature extractor
for layer in base_model.layers:
    layer.trainable = False


model = Sequential()
model.add(base_model)
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(1024,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(256,activation = 'relu'))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(4,activation = 'softmax'))
model.summary()
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 batch_normalization_1 (Bat  (None, 7, 7, 1280)        5120      
 chNormalization)                                                
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 3, 3, 1280)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 11520)             0         
                                                                 
 dense_5 (Dense)             (None, 1024)              11797504  
                                                      

In [16]:
model.compile(
    optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy']
)
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5),
    tf.keras.callbacks.ModelCheckpoint('model.h5',save_best_only=True),
    tf.keras.callbacks.TensorBoard(log_dir='logs')
]
results = model.fit(train_data, validation_data=valid_data, epochs=100, verbose=1)


Epoch 1/100
38/38 [==============================] - 48s 1s/step - loss: 2.0578 - accuracy: 0.4682 - val_loss: 1.1540 - val_accuracy: 0.4306
Epoch 2/100
38/38 [==============================] - 44s 1s/step - loss: 1.5264 - accuracy: 0.4849 - val_loss: 1.0334 - val_accuracy: 0.4861
Epoch 3/100
38/38 [==============================] - 45s 1s/step - loss: 1.1268 - accuracy: 0.5268 - val_loss: 1.2119 - val_accuracy: 0.4861
Epoch 4/100
38/38 [==============================] - 42s 1s/step - loss: 1.0290 - accuracy: 0.5535 - val_loss: 1.0783 - val_accuracy: 0.4583
Epoch 5/100
38/38 [==============================] - 43s 1s/step - loss: 0.9817 - accuracy: 0.5736 - val_loss: 0.8884 - val_accuracy: 0.5833
Epoch 6/100
38/38 [==============================] - 43s 1s/step - loss: 0.8127 - accuracy: 0.6421 - val_loss: 0.9379 - val_accuracy: 0.5556
Epoch 7/100
38/38 [==============================] - 39s 1s/step - loss: 0.7744 - accuracy: 0.6656 - val_loss: 0.9353 - val_accuracy: 0.5417
Epoch 8/100
3

In [18]:
loss, acc = model.evaluate(test_data,verbose = 1)

19/19 [==============================] - 13s 666ms/step - loss: 1.1372 - accuracy: 0.7138


In [20]:
model.save('/content/drive/MyDrive/ModelData')